## Fine tunning T5 Model on Our Data

In [1]:
import pandas as pd
import numpy as np
import sys
import csv
maxInt = sys.maxsize

while True:
    # decrease the maxInt value by factor 10 
    # as long as the OverflowError occurs.

    try:
        csv.field_size_limit(maxInt)
        break
    except OverflowError:
        maxInt = int(maxInt/10)

In [4]:
data=pd.read_csv('train_hindi_new.csv',engine='python')
data.dropna(inplace=True)

In [5]:
data=data.head(5000)

In [6]:
data

,headline,summary,article
3,रिम करेगा दो हजार कर्मचारियों की छंटनी,ब्लैकबेरी हैंडसेट बनाने वाली कनाडा की कंपनी रि...,ब्लैकबेरी हैंडसेट बनाने वाली कनाडा की कंपनी रि...
7,अमेरिका : राष्ट्रपति पद की उम्मीदवार हिलेरी क्...,श्रद्धांजलि सभा के दौरान अचानक खराब हुई हिलेरी...,अमेरिका में राष्ट्रपति पद के लिए डेमोक्रेटिक प...
8,देश के सर्वश्रेष्ठ स्पिनरों में से एक है हरभजन...,पूर्व भारतीय कप्तान अनिल कुंबले का मानना है कि...,पूर्व भारतीय क्रिकेट कप्तान अनिल कुंबले का मान...
9,इंडिगो के स्टाफ ने दिल्ली एयरपोर्ट पर पैसेंजर ...,इंडिगो के ग्राउंड स्टाफ ने यात्री से की हाथापा...,इंडिगो एयरलाइन के प्रेसीडेंट और डायरेक्टर आदित...
10,नव्या नवेली के साथ दोस्ती पर जावेद जाफरी के बे...,मीजान जाफरी ने किया खुलासा\r\nनव्या नंदा को बत...,संजय लीला भंसाली की फिल्म 'मलाल' से बॉलीवुड मे...
...,...,...,...
10645,केंद्रीय कर्मचारियों के महंगाई भत्ते में 6 % क...,सरकार ने महंगाई से परेशान केंद्रीय कर्मचारियों...,सरकार ने महंगाई से परेशान केंद्रीय कर्मचारियों...
10648,उत्तराखंड उप-चुनाव : 40 हजार वोटों से जीते विज...,उत्तराखंड के मुख्यमंत्री विजय बहुगुणा सितारगंज...,उत्तराखंड के मुख्यमंत्री विजय बहुगुणा सितारगंज...
10649,"इस्तीफे के पीछे कोई राजनीति नहीं, 95 साल की मा...",नजीब जंग से आज अरविंद केजरीवाल ने मुलाकात की\r...,दिल्ली के उपराज्यपाल रहे नजीब जंग ने पद से इस्...
10650,किसी भी तरह की लापरवाही के लिए अधिकारियों की ज...,उपराज्यपाल नजीब जंग ने की राष्ट्रीय राजधानी की...,उपराज्यपाल नजीब जंग ने गुरुवार को राष्ट्रीय रा...


In [7]:
import torch

import datasets
from datasets import Dataset
from datasets import load_metric

In [8]:
from transformers import AutoTokenizer

model_checkpoint ='ai4bharat/IndicBART'
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

pad_on_right = tokenizer.padding_side == "right"

In [9]:
max_input_length = 1000
max_target_length = 100

In [11]:
df=data
train = df[:4000]
valid = df[4000:4500]
test = df[4500:]
train.shape, valid.shape,test.shape

((4000, 3), (500, 3), (500, 3))

In [12]:
def preprocess_function(examples):
    inputs = ['summarize:' + doc for doc in examples["article"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True,padding='max_length')

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["summary"], max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [13]:
import torch
import datasets
from datasets import Dataset

train = Dataset.from_pandas(train)
valid = Dataset.from_pandas(valid)

In [14]:
tokenized_train = train.map(preprocess_function, batched=True)
tokenized_valid = valid.map(preprocess_function, batched=True)

Map:   0%|          | 0/4000 [00:00<?, ? examples/s]

C:\Users\Acer\anaconda3\lib\site-packages\transformers\tokenization_utils_base.py:3546: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/500 [00:00<?, ? examples/s]

In [15]:
import transformers
assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast)

### Fine Tuning

In [16]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

In [17]:
batch_size = 10
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [18]:
import nltk
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]
    
    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    # Extract a few results
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    
    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    
    return {k: round(v, 4) for k, v in result.items()}


In [19]:
import gc
gc.collect()

35

In [21]:
# determine the device we will be using for training
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
#print('There are %d GPU(s) available.' % torch.cuda.device_count())

In [22]:
torch.cuda.empty_cache()

In [23]:
%env WANDB_DISABLED=True

env: WANDB_DISABLED=True


In [24]:
model_name = model_checkpoint.split("/")[-1]
args = Seq2SeqTrainingArguments(
    f"{model_name}-finetuned-newsarticles",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=5,
    predict_with_generate=True,
    fp16=True
)

ValueError: FP16 Mixed precision training with AMP or APEX (`--fp16`) and FP16 half precision evaluation (`--fp16_full_eval`) can only be used on CUDA devices.

In [25]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_valid,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

NameError: name 'args' is not defined

In [ ]:
metric = load_metric("rouge")

<ipython-input-23-7154407f1f07>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("rouge")


In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
trainer.train()

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,No log,2.419704,37.667800,21.224600,33.623100,34.471600,19.000000
2,No log,2.354662,35.999300,21.400700,32.354300,34.273500,19.000000
3,No log,2.313812,36.559300,21.530800,34.156200,35.957100,19.000000
4,No log,2.290521,38.301800,22.241200,35.601600,36.530900,19.000000
5,No log,2.282169,38.301800,22.241200,35.601600,36.530900,19.000000


TrainOutput(global_step=50, training_loss=2.9805242919921877, metrics={'train_runtime': 33.1042, 'train_samples_per_second': 15.104, 'train_steps_per_second': 1.51, 'total_flos': 132169728000000.0, 'train_loss': 2.9805242919921877, 'epoch': 5.0})

In [ ]:
eval_dataset = Dataset.from_pandas(test)

In [ ]:
eval_dataset = eval_dataset.map(
                preprocess_function,
                batched=True)

Map:   0%|          | 0/15 [00:00<?, ? examples/s]

/usr/local/lib/python3.9/dist-packages/transformers/tokenization_utils_base.py:3596: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [ ]:
predict_results = trainer.predict(
            eval_dataset,max_length=300, num_beams=3)

In [ ]:
metrics = predict_results.metrics

In [ ]:
metrics

{'test_loss': 3.2294347286224365,
 'test_rouge1': 30.4451,
 'test_rouge2': 17.1221,
 'test_rougeL': 23.8949,
 'test_rougeLsum': 26.528,
 'test_gen_len': 79.6667,
 'test_runtime': 5.356,
 'test_samples_per_second': 2.801,
 'test_steps_per_second': 0.373}

In [ ]:
if args.predict_with_generate:
    predictions = tokenizer.batch_decode(predict_results.predictions, skip_special_tokens=True, clean_up_tokenization_spaces=True)
    predictions = [pred.strip() for pred in predictions]

In [ ]:
for i in predictions:
  print("--------")
  print(i)
  print("---x---")

--------
SP chief Akhilesh Yadav and BSP chief Mayawati announced the SP-BSP alliance by holding a joint press conference. Mayawati said that in the interest of the country we have kept the Lucknow Guest House scandal on the shore, we had also had a coalition, but due to some reasons we had to separate.
---x---
--------
potential monument friends have been selected, they have been selected by the inspection and vision committee so that tourist facilities can be developed on heritage monuments.
---x---
--------
the accused, who was working with Mohol Panchayat Samiti as an agricultural officer, was arrested on Wednesday when he was accepting a bribe from a person. police said that the complainant said that the complainantDattatre Bedge had contacted Balasaheb Bhikaji Babur to approve the proposal of his Agricultural Service Center.
---x---
--------
a memorandum letter was signed between the two sides to set up a large food park in Telangana.
---x---
--------
the budget has been passed a

In [ ]:
test['summary'][123]

'If a bitter boss continues to roam around in the office, then the risk of heart attack increases in a study conducted on two lakh employees.'

In [ ]:
predictions[-2]

'study conducted on two lakh employees revealed that researchers say that employees who are under pressure during work are more likely to have a heart attack than people living under low pressure. if the person is always under stress, then he increases the possibility of a heart attack, even though similar conclusions were revealed in a study, but there was mutual opposition to some things, this fact was confirmed by the latest conclusion.'